pso_nn_application.py

Source Code for Enhancing Neural Network Performance using Particle Swarm Optimization (PSO)

In [ ]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.exceptions import ConvergenceWarning
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Suppress warnings for cleaner output

In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

Load dataset

In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target

Standardize features

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Split into training and validation sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Traditional Grid Search CV

In [ ]:
print("\n--- Performing Grid Search ---")
param_grid = {
'hidden_layer_sizes': [(10,), (30,), (50,), (100,)],
'learning_rate_init': [0.001, 0.01, 0.1],
}
mlp = MLPClassifier(max_iter=200, solver='adam', random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model_grid = grid_search.best_estimator_
y_pred_grid = best_model_grid.predict(X_val)
acc_grid = accuracy_score(y_val, y_pred_grid)

print("Best GridSearch Params:", grid_search.best_params_)
print("GridSearch Validation Accuracy:", acc_grid)

Particle Swarm Optimization

In [ ]:
print("\n--- Performing Particle Swarm Optimization ---")
SWARM_SIZE = 10
DIMENSIONS = 2
INFORMANTS = 3
NUM_GENERATIONS = 20
W = 0.729
C1 = 1.49
C2 = 1.49
MIN_BOUNDARY = [0.0001, 5]
MAX_BOUNDARY = [0.1, 100]
desired_precision = 1e-5

def fitness_function(position):
lr = position[0]
hidden = int(position[1])
if hidden <= 0: hidden = 1
clf = MLPClassifier(hidden_layer_sizes=(hidden,), learning_rate_init=lr,
max_iter=200, solver='adam', random_state=42)
clf.fit(X_train, y_train)
pred = clf.predict(X_val)
acc = accuracy_score(y_val, pred)
return 1 - acc

class Particle:
def init(self):
self.position = [random.uniform(MIN_BOUNDARY[d], MAX_BOUNDARY[d]) for d in range(DIMENSIONS)]
self.velocity = [random.uniform(-1, 1) for _ in range(DIMENSIONS)]
self.fitness = fitness_function(self.position)
self.best_position = list(self.position)
self.best_fitness = self.fitness
self.informants = random.sample(range(SWARM_SIZE), INFORMANTS)
self.group_best_position = list(self.position)
self.group_best_fitness = self.fitness




In [ ]:
def update_velocity(self):
    for d in range(DIMENSIONS):
        r1, r2 = random.random(), random.random()
        cognitive = C1 * r1 * (self.best_position[d] - self.position[d])
        social = C2 * r2 * (self.group_best_position[d] - self.position[d])
        self.velocity[d] = W * self.velocity[d] + cognitive + social

def update_position(self):
    for d in range(DIMENSIONS):
        self.position[d] += self.velocity[d]
        self.position[d] = max(MIN_BOUNDARY[d], min(MAX_BOUNDARY[d], self.position[d]))
    self.fitness = fitness_function(self.position)

def update_group_best(self, swarm):
    best_informant = min(self.informants, key=lambda i: swarm[i].best_fitness)
    if swarm[best_informant].best_fitness < self.group_best_fitness:
        self.group_best_fitness = swarm[best_informant].best_fitness
        self.group_best_position = list(swarm[best_informant].best_position)

In [ ]:
swarm = [Particle() for _ in range(SWARM_SIZE)]
global_best = min(swarm, key=lambda p: p.best_fitness)
global_best_position = list(global_best.best_position)
global_best_fitness = global_best.best_fitness

pso_accuracies = []

PSO main loop

In [ ]:
for gen in range(NUM_GENERATIONS):
for particle in swarm:
particle.update_group_best(swarm)
particle.update_velocity()
particle.update_position()
if particle.fitness < particle.best_fitness:
particle.best_fitness = particle.fitness
particle.best_position = list(particle.position)

In [ ]:
best_particle = min(swarm, key=lambda p: p.best_fitness)
if best_particle.best_fitness < global_best_fitness:
    global_best_fitness = best_particle.best_fitness
    global_best_position = list(best_particle.best_position)

acc = 1 - global_best_fitness
pso_accuracies.append(acc)
print(f"Generation {gen+1}: Best Accuracy = {acc:.4f}")
if global_best_fitness < desired_precision:
    print("Desired precision reached.")
    break

In [ ]:
print("\nBest PSO Parameters:")
print("Learning Rate:", round(global_best_position[0], 5))
print("Hidden Neurons:", int(global_best_position[1]))
print("Validation Accuracy:", 1 - gl

Plot accuracy trend

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(pso_accuracies, label='PSO Accuracy', marker='o')
plt.axhline(y=acc_grid, color='r', linestyle='--', label='Grid Search Accuracy')
plt.title('PSO vs Grid Search Accuracy')
plt.xlabel('Generation')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('pso_vs_grid.png')
plt.show()